In [6]:
from pathlib import Path
import pandas as pd

# Load the test file from Data/Input into a DataFrame
base_dir = Path().resolve()  # current notebook directory: .../rag_excel_postgres/testing
file_path = base_dir.parent / "Data" / "preprocessing" / "efg_december_2025.csv"

df = pd.read_excel(file_path)  # header row is the second row in sheet
print(f"Loaded {len(df)} rows from {file_path}")
df.head()


ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [3]:
import os
import psycopg2
from psycopg2.extras import execute_values
from dotenv import load_dotenv

# Insert the loaded DataFrame into Postgres. First ensure a client row
# exists in `clients` with the chosen client_id and the source file name,
# then insert rows into `Mastersheet-Keyword_report` using that client_id.

load_dotenv()
DB_CONFIG = {
    "host": os.getenv("DB_HOST"),
    "port": int(os.getenv("DB_PORT")),
    "dbname": os.getenv("DB_NAME"),
    "user": os.getenv("DB_USER"),
    "password": os.getenv("DB_PASSWORD"),
}

TABLE_NAME = 'Mastersheet-Keyword_report'
CLIENT_ID = 1  # adjust if you need a different client id
CLIENT_NAME = file_path.stem  # use source file name as client name

# Align DataFrame columns to table schema
expected_cols = [
    "Keyword",
    "Initial ranking_month_year",
    "Current Rank_month_year",
    "Change +/-",
    "Search Volume",
    "Map Ranking (GBP)",
    "Location(state,country)",
    "URL",
    "Difficulty",
    "Search Intent",
]

missing = [c for c in expected_cols if c not in df.columns]
if missing:
    raise ValueError(f"Missing expected columns: {missing}")

# Clean rows and prepare payload
records = []
for _, row in df[expected_cols].iterrows():
    clean_row = row.where(pd.notna(row), None)
    records.append(
        (
            CLIENT_ID,
            clean_row["Keyword"],
            clean_row["Initial ranking_month_year"],
            clean_row["Current Rank_month_year"],
            clean_row["Change +/-"],
            clean_row["Search Volume"],
            clean_row["Map Ranking (GBP)"],
            clean_row["Location(state,country)"],
            clean_row["URL"],
            clean_row["Difficulty"],
            clean_row["Search Intent"],
        )
    )

insert_query = f"""
INSERT INTO "{TABLE_NAME}" (
    client_id,
    keyword,
    initial_ranking,
    current_ranking,
    change,
    search_volume,
    map_ranking_gbp,
    location,
    url,
    difficulty,
    search_intent
) VALUES %s
"""

conn = psycopg2.connect(**DB_CONFIG)
with conn:
    with conn.cursor() as cur:
        # Upsert client row first so foreign key is satisfied
        cur.execute(
            """
            INSERT INTO clients (client_id, client_name)
            VALUES (%s, %s)
            ON CONFLICT (client_id)
            DO UPDATE SET client_name = EXCLUDED.client_name;
            """,
            (CLIENT_ID, CLIENT_NAME),
        )

        # Insert keyword report rows
        execute_values(cur, insert_query, records)

        cur.execute("SELECT COUNT(*) FROM \"%s\"" % TABLE_NAME)
        total = cur.fetchone()[0]
        print(f"Inserted {len(records)} rows for client_id={CLIENT_ID}. Table now has {total} rows.")

conn.close()


Inserted 6 rows for client_id=1. Table now has 6 rows.
